In [1]:
import numpy as np
import warnings
warnings.filterwarnings('ignore')
import wandb
from pathlib import Path

from views_forecasts.extensions import *
from utils import fetch_data, transform_data, get_config_path, get_config_from_path, retrain_transformed_sweep, evaluate

# import importlib
# import utils  
# importlib.reload(utils)

In [ ]:
import os
os.environ['WANDB_SILENT'] = 'true'

In [2]:
def train():
    wandb.init(config=common_config)
    
    wandb.config.update(model_config, allow_val_change=True)
    retrain_transformed_sweep(Datasets_transformed)
    evaluate('calib', para_transformed)

In [3]:
level = 'cm'
config_path = Path('./my_config')

In [4]:
transforms = ['raw', 'log', 'normalize', 'standardize']
Datasets_transformed = {}
para_transformed = {}
qslist, Datasets = fetch_data(level)
for t in transforms:
    Datasets_transformed[t], para_transformed[t] = transform_data(Datasets, t)

Fetching and transforming data
 .     

In [5]:
common_config_path, wandb_config_path, model_config_path, sweep_config_path = get_config_path(config_path)
common_config = get_config_from_path(common_config_path, 'common')
wandb_config = get_config_from_path(wandb_config_path, 'wandb')

In [ ]:
for sweep_file in sweep_config_path.iterdir():
    if sweep_file.is_file():
        model_file = model_config_path / sweep_file.name
        if not model_file.is_file():
            raise FileNotFoundError(f'The corresponding model configuration file {model_file} does not exist.')

        sweep_config = get_config_from_path(sweep_file, 'sweep')
        model_config = get_config_from_path(model_file, 'model')

        sweep_id = wandb.sweep(sweep_config, project=wandb_config['project'],
                               entity=wandb_config['entity'])
        wandb.agent(sweep_id, function=train)

Create sweep with ID: tiqmg1o1
Sweep URL: https://wandb.ai/model-development-and-deployment/training_example_3/sweeps/tiqmg1o1


wandb: Agent Starting Run: fotzvjni with config:
wandb: 	learning_rate: 0.05
wandb: 	n_estimators: 100
wandb: 	n_jobs: 12
wandb: 	transform: log
wandb: Currently logged in as: xiaolong-sun (model-development-and-deployment). Use `wandb login --relogin` to force relogin


Training model fatalities003_nl_baseline_rf
Calibration partition (log)
 * == Performing a run: "fatalities003_nl_baseline_rf_calib_log" == * 
Model object named "fatalities003_nl_baseline_rf_calib_log" with equivalent metadata already exists.
Retrain is true, overwriting "fatalities003_nl_baseline_rf_calib_log"
Training model(s)...
Storing "fatalities003_nl_baseline_rf_calib_log"
Getting predictions
pr_56_cm_fatalities003_nl_baseline_rf_calib_log_estimators_100_lr_0.05.parquet
Test partition ({transform})
 * == Performing a run: "fatalities003_nl_baseline_rf_test_log" == * 
Model object named "fatalities003_nl_baseline_rf_test_log" with equivalent metadata already exists.
Retrain is true, overwriting "fatalities003_nl_baseline_rf_test_log"
Training model(s)...
Storing "fatalities003_nl_baseline_rf_test_log"
Getting predictions
pr_56_cm_fatalities003_nl_baseline_rf_test_log_estimators_100_lr_0.05.parquet
**************************************************************
Evaluating model fa

mse,▁
mse,230570.73604


wandb: Agent Starting Run: qpy7zmr6 with config:
wandb: 	learning_rate: 0.05
wandb: 	n_estimators: 100
wandb: 	n_jobs: 12
wandb: 	transform: standardize


...

mse,▁
mse,23.24465


wandb: Agent Starting Run: m3nce8af with config:
wandb: 	learning_rate: 0.2
wandb: 	n_estimators: 100
wandb: 	n_jobs: 12
wandb: 	transform: log


Training model fatalities003_joint_narrow_xgb
Calibration partition (log)
 * == Performing a run: "fatalities003_joint_narrow_xgb_calib_log" == * 
Model object named "fatalities003_joint_narrow_xgb_calib_log" with equivalent metadata already exists.
Retrain is true, overwriting "fatalities003_joint_narrow_xgb_calib_log"
Training model(s)...


Reordering feature dimension. Save memory by setting the outcome feature as the first column in your dataframe.


Storing "fatalities003_joint_narrow_xgb_calib_log"
Getting predictions
pr_56_cm_fatalities003_joint_narrow_xgb_calib_log_estimators_100_lr_0.2.parquet
cm_fatalities003_joint_narrow_xgb_calib_log_estimators_100_lr_0.2 , run Fatalities003 does not exist, predicting
